In [20]:
#saving path to repository and importing libraries
repo_path = "./titanic"

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mtpl

In [22]:
#loading training data
train_data_path = "{}/train/train.tsv".format(repo_path)
train_data = pd.read_csv(train_data_path, header=0,delimiter='\t')


In [23]:
#prediction of surviving is gonna be based on passenger's sex and class
X_columns = ['Sex','Pclass']
y_column = ['Survived']


In [24]:
#maping string values to numbers
replace_dict = {'male': 0, 'female' : 1} 

train_data['Sex']=train_data['Sex'].map(replace_dict)



In [25]:
X = train_data[X_columns].values
y = train_data[y_column].values.flatten()


In [26]:
#using KNeighborsClassifier from sklearn as a model
from sklearn.neighbors import KNeighborsClassifier

In [27]:
model = KNeighborsClassifier(n_neighbors=2)

In [28]:
#training the model
X = train_data[X_columns]
y = train_data['Survived']

model.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [29]:
#loading, preparing and saving valid and test data
valid_data_X = pd.read_csv("{}/dev-0/in.tsv".format(repo_path),
                           delimiter='\t',
                           names=train_data.columns[1:])

valid_data_y = pd.read_csv("{}/dev-0/expected.tsv".format(repo_path),
                           delimiter='\t',
                           names=train_data.columns[:1])

In [30]:
test_data_X = pd.read_csv("{}/test-A/in.tsv".format(repo_path),
                           delimiter='\t',
                           names=train_data.columns[1:])

In [31]:

replace_dict = {'male': 0, 'female' : 1} 

valid_data_X['Sex']=valid_data_X['Sex'].map(replace_dict)


test_data_X['Sex']=test_data_X['Sex'].map(replace_dict)



In [32]:
X_valid = valid_data_X[X_columns].values
y_valid = valid_data_y[y_column].values
X_test = test_data_X[X_columns].values

In [33]:
#using trained model for valid data
valid_predicted = model.predict(X_valid)

In [34]:
#counting valid results' accuracy
from sklearn.metrics import accuracy_score

print(accuracy_score(y_valid, valid_predicted))

0.7611940298507462


In [35]:
#finding k with the highest accuracy for valid data
from sklearn.model_selection import cross_val_score

neighbours = []
cv_scores=[]
for i in range(1,100):
    neighbours.append(i)
for item in neighbours:
    knn = KNeighborsClassifier(n_neighbors=item)
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

max = 0
maxi=0
for i in range(0,len(cv_scores)):
    if cv_scores[i]>max:
        max = cv_scores[i]
        maxi=i

print("k with the highest result is: ", neighbours[maxi])

k with the highest result is:  19


In [36]:
#using trained model for test data
test_predicted = model.predict(X_test)

In [37]:
#saving results in new files
pd.Series(valid_predicted.flatten()).to_csv("{}/dev-0/out.tsv".format(repo_path),
                                            sep='\t',
                                            index=None)

pd.Series(test_predicted.flatten()).to_csv("{}/test-A/out.tsv".format(repo_path),
                                           sep='\t',
                                           index=None)